In [234]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [235]:
# Create the optimization model
model = gb.Model("Question 1: Vaccines")

In [236]:
Billy_Bishop_Toronto_City_Airport_costs = [0.05, 0.05, 0.05, 0.05, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07, 0.07, 0.07, 0.07, 0.07, 0.08, 0.08, 0.08, 0.08, 0.08, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1]
Toronto_Pearson_Airport_costs = [0.08, 0.08, 0.08, 0.08, 0.08, 0.05, 0.05, 0.05, 0.05, 0.05, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1, 0.1, 0.07, 0.07, 0.07, 0.07, 0.07, 0.06, 0.06, 0.06, 0.06]

In [237]:
# Create the a single class of decision variables where
# From = {Billy} and To = {29 sites}.
x = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Billy Shipping")
# From = {Pearson} and To = {29 sites}.
y = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Pearson Shipping")

In [238]:
Billy_objective = gb.quicksum(Billy_Bishop_Toronto_City_Airport_costs[j] * x[j] for j in range(29))
Pearson_objective = gb.quicksum(Toronto_Pearson_Airport_costs[j] * y[j] for j in range(29))
model.setObjective(Billy_objective + Pearson_objective, GRB.MINIMIZE)

In [239]:
model.addConstr(gb.quicksum(x[j] for j in range(5)) - gb.quicksum(y[j] for j in range(5)) <= 4800, name="4800 constraint")
model.addConstr(gb.quicksum(y[j] for j in range(5)) - gb.quicksum(x[j] for j in range(5)) <= 4800, name="4800 constraint negative")

<gurobi.Constr *Awaiting Model Update*>

In [240]:
model.addConstr(gb.quicksum(y[j] for j in range(20,25)) <= 8 * (gb.quicksum(x[j] for j in range(10,15))), name="Second constraint")

<gurobi.Constr *Awaiting Model Update*>

In [241]:
model.addConstr(gb.quicksum(x[j] for j in range(25,29)) >= 0.8 * (gb.quicksum(y[j] for j in range(15,20))), name="Third constraint")

<gurobi.Constr *Awaiting Model Update*>

In [242]:
model.addConstr(gb.quicksum(x[j] for j in range(29)) == 100000, "Billy Supply Constraint")
model.addConstr(gb.quicksum(y[j] for j in range(29)) == 250000, "Pearson Supply Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [243]:
#Hosipital Constraint
#model.addConstr((gb.quicksum(x[j] + y[j] for j in range(7))) == 4 * (gb.quicksum(x[j] + y[j] for j in range(7,29))), "Hosipital Constraint")

In [244]:
#Hosipital Constraint
# 50000 = 7x + 22x/4
# x = 4000
# 4000 per day, 7 days = 28000 per week
for j in range(7):
    model.addConstr(x[j] <= 28000, f"Hospital Billy Constraint_{j}")
for j in range(7):
    model.addConstr(y[j] <= 28000, f"Hospital Pearson Constraint_{j}")

# 50000 = 4*7x + 22x
# x = 1000
# 1000 per day, 7 days = 7000 per week
for j in range(7,29):
    model.addConstr(x[j] <= 7000, f"Non-Hospital Billy Constraint_{j}")
for j in range(7,29):
    model.addConstr(y[j] <= 7000, f"Non-Hospital Pearson Constraint_{j}")

In [245]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 64 rows, 58 columns and 155 nonzeros
Model fingerprint: 0x330b2b8b
Coefficient statistics:
  Matrix range     [8e-01, 8e+00]
  Objective range  [5e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 3e+05]
Presolve removed 61 rows and 52 columns
Presolve time: 0.01s
Presolved: 3 rows, 7 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1867500e+04   1.196875e+03   0.000000e+00      0s
       1    2.1963250e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.196325000e+04


In [246]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Billy Shipping[0]        28000 
Billy Shipping[1]        28000 
Billy Shipping[3]        28000 
Billy Shipping[4]        11625 
Billy Shipping[14]         4375 
Pearson Shipping[0]        28000 
Pearson Shipping[2]        28000 
Pearson Shipping[3]        28000 
Pearson Shipping[4]        16425 
Pearson Shipping[5]        28000 
Pearson Shipping[6]        28000 
Pearson Shipping[7]         7000 
Pearson Shipping[8]         7000 
Pearson Shipping[9]         7000 
Pearson Shipping[13]         7000 
Pearson Shipping[14]         2575 
Pearson Shipping[20]         7000 
Pearson Shipping[21]         7000 
Pearson Shipping[22]         7000 
Pearson Shipping[23]         7000 
Pearson Shipping[24]         7000 
Pearson Shipping[25]         7000 
Pearson Shipping[26]         7000 
Pearson Shipping[27]         7000 
Pearson Shipping[28]         7000 
None


In [247]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  21963.25
